# Data Combine의 다른 지역으로 수정한 버전

* 최종적으로 만들어지는 **csv파일**은 Git에 올리면 용량 크다고 에러 나니까 
* 저장해놓고 **업로드(Stage & Push) 하지 말것**

In [3]:
import pandas as pd

# 읍면동에 대한 데이터 불러오기
df = pd.read_csv('dataset/Chuncheon/춘천 읍면동.csv', encoding='cp949')
emds = list(df['읍면동'])

# 데이터 종류에 대한 데이터
types = ['강수','기온','뇌전','습도','풍속','풍향','하늘상태']

# 데이터 연도에 대한 데이터
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']

dfs = []
# 읍면동을 차례로 불러오기
for emd in emds:
    dfs2 = []

    # 연도 데이터를 차례로 불러오기
    for year in years:

        # 데이터 종류를 차례로 불러오기
        for datatype in types: 
            
            # 2010년은 6월부터, 2022년은 4월까지만의 데이터가 존재하므로 아래와 같이 파일명 구분
            if year == '2010':
                df_temp = pd.read_csv('dataset/Chuncheon/'+emd+'_'+datatype+'_'+year+'06_'+year+'12.csv')
            elif year == '2022':
                df_temp = pd.read_csv('dataset/Chuncheon/'+emd+'_'+datatype+'_'+year+'01_'+year+'04.csv')
            else:
                df_temp = pd.read_csv('dataset/Chuncheon/'+emd+'_'+datatype+'_'+year+'01_'+year+'12.csv')
            
            # 컬럼명을 통일시켜줌
            df_temp.columns = ['day','hour',datatype]

            # 강수량에 대한 데이터프레임을 기준으로 나머지 데이터의 컬럼을 추가
            if datatype == '강수':
                df = df_temp
            else:
                df[datatype] = df_temp[datatype]

        # 원본 데이터에서 월을 구분하는 부분은 첫행을 제외하고 나머지 부분에 None이 저장되어있음
        # 해당하는 행 번호를 추출하여 월 구분 및 이후 해당 행 제거 할 때 사용
        del_row = list(df[df.iloc[:,2].isna()].index)

        # 데이터에 몇 월의 데이터인지를 추가
        # 이때 2010년의 경우 6월부터 데이터가 존재하므로 아래와 같이 구분
        if year == '2010':
            j=0
            for n,i in enumerate(del_row + [len(df)-1]):
                df.loc[j:i,'month'] = n+6
                j=i+1
        else:
            j=0
            for n,i in enumerate(del_row + [len(df)-1]):
                df.loc[j:i,'month'] = n+1
                j=i+1

        # 월이 적혀있는 필요없는 행 제거
        df.drop(del_row, axis=0, inplace=True)
        df['year'] = int(year)

        # 연도별로 구분된 데이터를 dfs2에 추가
        dfs2.append(df)
    
    # dfs2에 저장된 연도별 데이터를 병합
    df2 = pd.concat(dfs2, ignore_index=True)

    # 읍면동에 해당하는 컬럼을 추가
    df2['읍면동'] = emd

    # 읍면동별로 구분된 데이터를 dfs에 추가 
    dfs.append(df2)

# dfs에 존재하는 데이터를 최종적으로 병합
df_final = pd.concat(dfs, ignore_index=True)


In [4]:
df_final

,day,hour,강수,기온,뇌전,습도,풍속,풍향,하늘상태,month,year,읍면동
0,1,0.0,0.0,14.300000,-1.0,47.0,52.799999,389.0,1.0,6.0,2010,강남동
1,1,100.0,0.0,17.200001,-1.0,37.0,52.799999,389.0,1.0,6.0,2010,강남동
2,1,200.0,0.0,20.299999,-1.0,29.0,52.799999,389.0,1.0,6.0,2010,강남동
3,1,300.0,0.0,21.700001,-1.0,25.0,52.799999,389.0,1.0,6.0,2010,강남동
4,1,400.0,0.0,23.700001,-1.0,23.0,52.799999,389.0,1.0,6.0,2010,강남동
...,...,...,...,...,...,...,...,...,...,...,...,...
2715643,30,1900.0,0.0,4.200000,NaN,97.0,0.400000,78.0,NaN,4.0,2022,후평3동
2715644,30,2000.0,0.0,3.900000,NaN,98.0,0.500000,317.0,NaN,4.0,2022,후평3동
2715645,30,2100.0,0.0,5.600000,NaN,95.0,0.900000,318.0,NaN,4.0,2022,후평3동
2715646,30,2200.0,0.0,7.700000,NaN,85.0,1.100000,316.0,NaN,4.0,2022,후평3동


In [5]:
# 읍면동의 수 확인
len(list(df_final['읍면동'].unique()))

26

In [6]:
df_final.rename(columns = {'강수' : 'rainfall','기온' : 'temp','뇌전' : 'lightning', '습도':'humidity', '풍향':'wind_direction','풍속':'wind_speed','하늘상태':'condition','읍면동':'region'}, inplace = True)
df_final

,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,month,year,region
0,1,0.0,0.0,14.300000,-1.0,47.0,52.799999,389.0,1.0,6.0,2010,강남동
1,1,100.0,0.0,17.200001,-1.0,37.0,52.799999,389.0,1.0,6.0,2010,강남동
2,1,200.0,0.0,20.299999,-1.0,29.0,52.799999,389.0,1.0,6.0,2010,강남동
3,1,300.0,0.0,21.700001,-1.0,25.0,52.799999,389.0,1.0,6.0,2010,강남동
4,1,400.0,0.0,23.700001,-1.0,23.0,52.799999,389.0,1.0,6.0,2010,강남동
...,...,...,...,...,...,...,...,...,...,...,...,...
2715643,30,1900.0,0.0,4.200000,NaN,97.0,0.400000,78.0,NaN,4.0,2022,후평3동
2715644,30,2000.0,0.0,3.900000,NaN,98.0,0.500000,317.0,NaN,4.0,2022,후평3동
2715645,30,2100.0,0.0,5.600000,NaN,95.0,0.900000,318.0,NaN,4.0,2022,후평3동
2715646,30,2200.0,0.0,7.700000,NaN,85.0,1.100000,316.0,NaN,4.0,2022,후평3동


In [7]:
len(list(df_final['rainfall'].unique()))

340

In [8]:
df_final.to_csv("output/Chuncheon.csv",mode='w',index=False,encoding='cp949')

In [9]:
del df_final

: 